In [2]:
from manim import *
import numpy as np

Manim Community v0.17.3

In [3]:
from random import uniform, seed

class Abertura(Scene):

    def construct(self):
        seed(0xDEADBEEF)

        self.camera.background_color = '#191919'

        main_dot = Dot(radius = .35)

        group_dots = VGroup(
            *[Dot(radius = .01).shift(i * 0.3 * RIGHT * uniform(-1, 1) + i * UP * 0.2 * uniform(-1, 1)) 
              for i in range(0, 150)]
              )
        
        colors = [BLUE_A, BLUE_B, BLUE_C, BLUE_D, BLUE_E]

        ellipses = VGroup(
            *[Ellipse(color = colors[i], width = 3, height = 1).rotate(i * PI / 4)
              for i in range(5)]
        )

        ellipse_final = Ellipse(color = WHITE, width = 1.5, height = .5).rotate(PI / 4)

        name = Text('Sentido Físico', font = 'Bebas Neue').shift(DOWN)

        self.play(GrowFromCenter(main_dot), run_time = .5)
        self.play(Write(ellipses), run_time = 1.7)
        self.play(ReplacementTransform(ellipses, ellipse_final), run_time = 1)
        self.play(ellipse_final.animate.surround(main_dot), FadeIn(group_dots), run_time = .5)
        self.play(Write(name, run_time = 1))
        self.wait(.5)
        self.play(FadeOut(group_dots), Uncreate(main_dot), Uncreate(ellipse_final), FadeOut(name))
        self.wait(.5)

In [4]:
%manim -qh -v WARNING Abertura

In [5]:
%manim -qm -v WARNING Cena1

# Configuração Normal Full HD
config.frame_width = 16
config.frame_height = 9

config.pixel_width = 1920
config.pixel_height = 1080

# Configuração para stories
#config.frame_width = 9
#config.frame_height = 16

#config.pixel_width = 1080
#config.pixel_height = 1920

class Cena1(Scene):

    def construct(self):

        self.camera.background_color = '#191919'

        ax = Axes([-3, 3, 1], [-3, 3, 1], tips = False)
        circle = Circle(color = WHITE)
        sine = ax.plot(lambda x: np.sin(2 * x), color = RED_C) # Construir a função seno utilizando o método .plot() e a função lambda
        area1 = ax.get_area(sine, x_range = [-PI / 2, PI / 2])
        func = MathTex('f(x) = sin(2x)').shift(UP * 2 + RIGHT * 3)
        func2 = MathTex('f(x) = cos(2x)').shift(UP * 2 + RIGHT * 3)
        cos = ax.plot(lambda x: np.sin(2 * x + PI / 2), color = BLUE_C)
        area2 = ax.get_area(cos, x_range = [-PI / 2, PI / 2])

        self.play(Write(ax, run_time = 2), Write(circle, run_time = 1.5))
        self.play(circle.animate.scale(2))
        self.play(ReplacementTransform(circle, sine, run_time = 2), Write(func, run_time = 1))
        self.play(Create(area1))
        self.play(Uncreate(area1))
        self.play(ReplacementTransform(sine, cos), Transform(func, func2, run_time = .5))
        self.play(Create(area2))
        self.wait(2)

In [10]:
%%manim -ql -v WARNING AnimGrupo

from manim.utils.color import Color

class AnimGrupo(Scene):

    def construct(self):

        self.camera.background_color = '#191919'

        # Criar um grupo de quadrados e dispô-los em um grid
        squares = VGroup(*[Square(color = Color(hue = j/50, saturation = .5, 
        luminance = .5), fill_opacity = .5) for j in range(20)]).arrange_in_grid(4, 5).scale(.5)

        self.play(AnimationGroup(*[FadeIn(s) for s in squares], lag_ratio = .2)) # Agrupar as animações com um delay de 0.2 s entre elas
        self.wait()

In [13]:
%%manim -qm -v WARNING AllUpdaterTypes

class AllUpdaterTypes(Scene):
    def construct(self):

        red_dot = Dot(color = RED).shift(LEFT)
        pointer = Arrow(ORIGIN, RIGHT).next_to(red_dot, LEFT)

        pointer.add_updater(
            lambda mobject: mobject.next_to(red_dot, LEFT)
        )

        def shifter(mob, dt):
            mob.shift(2 * RIGHT * dt)
        
        red_dot.add_updater(shifter)

        def scene_scaler(dt):
            for mob in self.mobjects:
                mob.set(width = 2 / (1 + np.linalg.norm(mob.get_center())))
        
        self.add_updater(scene_scaler)
        
        self.add(red_dot, pointer)
        self.update_self(0)
        self.wait(5)

In [16]:
%%manim -qm -v WARNING UpdaterAndAnimation

class UpdaterAndAnimation(Scene):
    def construct(self):

        red_dot = Dot(color = RED).shift(LEFT)
        rotating_square = Square()

        rotating_square.add_updater(
            lambda mobject, dt: mobject.rotate(PI * dt)
        )

        def shifter(mob, dt):
            mob.shift(2 * RIGHT * dt)
        red_dot.add_updater(shifter)

        self.add(red_dot, rotating_square)
        self.wait()
        red_dot.suspend_updating()
        self.wait()

        self.play(
            red_dot.animate.shift(UP),
            rotating_square.animate.move_to([-2, 2, 0])
        )

        self.wait(2)

In [24]:
%%manim -qm -v WARNING ValueTrackerEx1

class ValueTrackerEx1(Scene):
    def construct(self):
        line = NumberLine(x_range = [-5, 5])
        position = ValueTracker(0)
        pointer = Vector(DOWN)

        pointer.add_updater(
            lambda mob: mob.next_to(
            line.number_to_point(position.get_value()), UP
        ))

        pointer.update()
        self.play(Write(line))
        self.play(FadeIn(pointer))
        self.wait()
        self.play(position.animate.set_value(4))
        self.wait()
        self.play(position.animate.set_value(-2))
        self.wait()

In [50]:
%%manim -qm -v WARNING UpdateFunction

class UpdateFunction(Scene):
    def construct(self):

        s = ValueTracker(1)
        ax = Axes(x_range = [-4, 4, 1], y_range = [-4, 4, 1], tips = False)
        sine = ax.plot(lambda x: np.sin(x), color = BLUE_C)
        math = MathTex('f(x) = sin(x)').move_to(2.5 * UR)

        sine.add_updater(
            lambda mob: mob.become(
            ax.plot(lambda x: np.sin(s.get_value() * x), color = RED)
                )
        )

        s_number = DecimalNumber(s.get_value(), 
                                 color = RED, 
                                 num_decimal_places = 1, 
                                 show_ellipsis = False)
        
        s_number.add_updater(
            lambda mob: mob.set_value(s.get_value()).next_to(sine)
        )

        self.play(Write(ax)) 
        self.play(Write(sine), Write(math))
        self.play(FadeIn(s_number))
        self.play(s.animate.set_value(4))
        self.wait()
        self.play(s.animate.set_value(2))
        self.wait()
        self.play(s.animate.set_value(1 / 2))
        self.wait()

In [7]:
%%manim -qh -v WARNING AberturaIntense

class AberturaIntense(Scene):
    def construct(self):

        self.camera.background_color = '#181818'
        Text.set_default(font = 'Squada One')

        lamp = SVGMobject('svg\light-bulb.svg', fill_color = WHITE, fill_opacity = .3).scale(2)
        eqs = SVGMobject('eqs.svg', fill_color = GRAY_B, fill_opacity = .1).scale(8)
        
        tex = Text('INTENSE\n ENSINA', 
                    font_size = 65, weight = BOLD, 
                    t2c = {'[2:5]': '#FE7D10'}).move_to(lamp.get_center())

        self.play(Write(lamp, run_time = 2), 
                        Create(eqs, run_time = .7))

        self.play(Create(tex))
        self.play(Flash(tex, color = '#FE7D10', flash_radius = 2, line_length = .5, num_lines = 20, run_time = .8), 
                  ShowPassingFlash(tex.copy(), run_time = 1.2))

        self.wait(.5)
        self.play(Unwrite(tex), Unwrite(lamp), Uncreate(eqs))
        self.wait()